In [1]:
%cd /scratch/bruingjde/SNAM2021-code/

import os
import typing

import joblib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import sklearn.linear_model
import sklearn.model_selection
import sklearn.metrics
import sklearn.pipeline
import sklearn.preprocessing
from tqdm.auto import tqdm

import tlp

/scratch/bruingjde/SNAM2021-code


In [53]:
# Check
for entry in os.scandir('data'):
  if not os.path.isfile(os.path.join(entry, 'features', 't.pkl')): print(entry)

In [51]:
def t(path: str, verbose: bool = False):
  # Check if file exists
  feature_path = os.path.join(path, 'features')
  file = os.path.join(feature_path, 't.pkl')
  if os.path.isfile(file): return 
  
  os.makedirs(feature_path, exist_ok=True)

  # Read in
  edgelist_mature_file = os.path.join(path, 'edgelist_mature.pkl')
  if verbose: print(f'Read {edgelist_mature_file}')
  edgelist_mature = pd.read_pickle(edgelist_mature_file)
  
  instances_file = os.path.join(path, 'instances_sampled.npy')
  if verbose: print(f'Read {instances_file}')  
  instances_sampled = np.load(instances_file)

  nodes = {node for instance in instances_sampled for node in instance}
  
  G = nx.from_pandas_edgelist(edgelist_mature)
  
  # Calculate per node
  traingles_count = nx.triangles(G, nodes)
  
  triangles_values = [(traingles_count[u], traingles_count[v]) for u, v in instances_sampled]
  
  def diff(x): return abs(x[1]-x[0]) 
  
  results = {
    tlp.Experiment(feature='T', time_aware=False, nodepair_strategy=strategy_str): (
      np.array([strategy_func(instance) for instance in triangles_values])) 
    for strategy_str, strategy_func in {'sum': sum, 'diff': diff, 'max': max, 'min': min}.items()
  }
  
  joblib.dump(results, os.path.join(feature_path, 't.pkl'))

In [52]:
t('data/27', verbose=True)

Read data/27/edgelist_mature.pkl
Read data/27/instances_sampled.npy


In [ ]:
entries = sorted(os.scandir('data'), key=lambda x: x.name)
tlp.ProgressParallel(n_jobs=len(entries), total=len(entries))(
  joblib.delayed(t)(entry.path) for entry in entries
)